In [1]:
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


# ETL Using Pandas

In [15]:
# MySQL connection details
username = "root"
password = "root"
host = "localhost"
port = "3306"
database = "companydb"

# Create a connection to MySQL database
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# Read data from 'Employee' table of Data base & then convert it into a Pandas DataFrame
df = pd.read_sql('SELECT * FROM Employee', con=engine)

print(f"Extracted data fro MySQL is \n {df}")

#df.columns
# Ensure date columns are in datetime format
date_columns = ['JoiningDate', 'Employee_Last_Working_Date']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')  

# Required columns for transformation
required_columns = {'Salary', 'JoiningDate', 'Employee_Last_Working_Date'}
new_columns = {'Annual_CTC', 'Yearly_Performance_Bonus', 'Total_Months_Worked', 'Tax_Pay_Anam'}

# Check if required columns exist
if required_columns.issubset(df.columns):
    # Only add new columns if they are missing
    missing_new_columns = new_columns - set(df.columns)
    if missing_new_columns:
        if 'Salary' in df.columns:
            df['Annual_CTC'] = df['Salary'] * 12
            df['Yearly_Performance_Bonus'] = df['Annual_CTC'] * 1.15

        if {'JoiningDate', 'Employee_Last_Working_Date'}.issubset(df.columns):
            df['Total_Months_Worked'] = ((df['Employee_Last_Working_Date'] - df['JoiningDate']).dt.days // 30)

        if {'Annual_CTC', 'Yearly_Performance_Bonus'}.issubset(df.columns):
            df['Tax_Pay_Anam'] = (df['Annual_CTC'] + df['Yearly_Performance_Bonus']) * 0.28
       
        print(df)
        
        # Save data with correct date type
        df.to_sql('employee', con=engine, if_exists='replace', index=False, 
                  dtype={'JoiningDate': sqlalchemy.Date, 'Employee_Last_Working_Date': sqlalchemy.Date})
        df.to_csv('cleaned_data.csv', index=False)

        print("Data transformation and loading completed successfully.")
    else:
        print("New columns already exist. No transformations applied.")
else:
    print(f"Missing required columns: {required_columns - set(df.columns)}. No transformations applied.")

Extracted data fro MySQL is 
     EmployeeID    Name  DepID   Salary  Age JoiningDate  \
0            5    Emma      1  52000.0   32  2021-03-12   
1            6   Frank      2  63000.0   29  2018-11-20   
2            7   Grace      3  58000.0   33  2017-07-05   
3            8  Hannah      1  49000.0   27  2022-01-30   
4            9   Isaac      2  75000.0   42  2014-06-22   
5           10    Jack      3  57000.0   31  2019-02-15   
6           11   Karen      1  51000.0   29  2020-09-10   
7           12     Leo      2  72000.0   38  2016-12-05   
8           13     Mia      3  75000.0   36  2015-04-25   
9           14    Noah      1  53000.0   30  2021-07-14   
10          15  Olivia      2  68000.0   34  2017-10-03   
11          16    Paul      3  60000.0   40  2014-08-19   
12          17   Quinn      1  50000.0   28  2023-05-22   
13          18  Rachel      2  71000.0   39  2015-11-07   
14          19   Steve      3  56000.0   32  2019-06-30   
15          20   Tracy    

In [16]:
query = """
SELECT COLUMN_NAME, DATA_TYPE 
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_SCHEMA = 'companydb' AND TABLE_NAME = 'Employee'
"""

df_schema = pd.read_sql(query, con=engine)

df_schema

,COLUMN_NAME,DATA_TYPE
0,EmployeeID,bigint
1,Name,text
2,DepID,bigint
3,Salary,double
4,Age,bigint
5,JoiningDate,date
6,Employee_Last_Working_Date,date
7,manager_id,double
8,Annual_CTC,double
9,Yearly_Performance_Bonus,double


In [6]:
df.columns

Index(['EmployeeID', 'Name', 'DepID', 'Salary', 'Age', 'JoiningDate',
       'Employee_Last_Working_Date', 'manager_id', 'Annual_CTC',
       'Yearly_Performance_Bonus', 'Total_Months_Worked', 'Tax_Pay_Anam'],
      dtype='object')

df

# ETL Pipeline Using Python.

In [20]:
import pandas as pd
import pymysql
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime

# MySQL Connection Details
USERNAME = "root"
PASSWORD = "root"
HOST = "localhost"
PORT = "3306"
DATABASE = "companydb"

# Establish Database Connection
def create_db_connection():
    try:
        engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
        print("✅ Database Connection Successfully Initiate!")
        return engine
    except Exception as e:
        print(f"❌ Error connecting to database: {e}")
        return None

# Extract Data from MySQL
def extract_data(engine):
    try:
        query = "SELECT * FROM Employee"
        df = pd.read_sql(query, con=engine)
        print("✅ Data Extraction Successful!")
        return df
    except Exception as e:
        print(f"❌ Error extracting data: {e}")
        return None

# Transform Data (Adding New Columns)
def transform_data(df):
    if df is None or df.empty:
        print("❌ No data to transform.")
        return None

    # Convert Date Columns to DateTime
    date_columns = ['JoiningDate', 'Employee_Last_Working_Date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Define required and new columns
    required_columns = {'Salary', 'JoiningDate', 'Employee_Last_Working_Date'}
    new_columns = {'Annual_CTC', 'Yearly_Performance_Bonus', 'Total_Months_Worked', 'Tax_Pay_Anam'}

    if required_columns.issubset(df.columns):
        # Check if transformation columns exist, if not, apply transformations
        missing_new_columns = new_columns - set(df.columns)

        if missing_new_columns:
            print("🔄 Applying Transformations...")
            df['Annual_CTC'] = df['Salary'] * 12
            df['Yearly_Performance_Bonus'] = df['Annual_CTC'] * 1.15
            df['Total_Months_Worked'] = ((df['Employee_Last_Working_Date'] - df['JoiningDate']).dt.days // 30)
            df['Tax_Pay_Anam'] = (df['Annual_CTC'] + df['Yearly_Performance_Bonus']) * 0.28

            print("✅ Data Transformation Successful!")
        else:
            print("✅ No new transformations needed, data already processed.")

        return df
    else:
        print(f"❌ Missing required columns: {required_columns - set(df.columns)}. Transformation skipped.")
        return None

# Load Data Back into MySQL & Save as CSV
def load_data(df, engine):
    if df is None:
        print("❌ No data to load.")
        return

    try:
        df.to_sql('employee', con=engine, if_exists='replace', index=False,
                  dtype={'JoiningDate': sqlalchemy.Date, 'Employee_Last_Working_Date': sqlalchemy.Date})
        df.to_csv('cleaned_data.csv', index=False)

        print("✅ Data Loading Completed Successfully!")
    except Exception as e:
        print(f"❌ Error loading data: {e}")

# Main Function to Run ETL Pipeline
def run_etl_pipeline():
    engine = create_db_connection()
    if engine:
        df = extract_data(engine)
        df_transformed = transform_data(df)
        load_data(df_transformed, engine)

# Run ETL Pipeline
if __name__ == "__main__":
    run_etl_pipeline()


✅ Database Connection Successfully Initiate!
✅ Data Extraction Successful!
✅ No new transformations needed, data already processed.
✅ Data Loading Completed Successfully!
